# Voxelize

In [1]:
from voxelize import *

data_path = '/home/jiazx_ug/dataset/S4/O/training/velodyne/335422.bin'
label_path = '/home/jiazx_ug/dataset/S4/O/training/label_2/335422.txt'

vsize_xyz = np.array([0.5, 0.5, 1])       
coors_range_xyz = np.array([-70.4, -40, -3, 70.4, 40, 1])  
num_point_features = 4                                 
max_num_points_per_voxel = 5               
max_num_voxels = 16000
input_channels = 4

grid_size = get_grid_size(coors_range_xyz, vsize_xyz)
backbone = input_backbone(input_channels, grid_size)
checker = isPointInQuadrangle()

In [2]:
points = read_data(data_path)
voxels, voxel_indices, num_points = create_voxel(points, vsize_xyz, coors_range_xyz,num_point_features ,max_num_points_per_voxel, max_num_voxels)
input_feature, input_indices = get_input_feature(voxels, num_points, voxel_indices, backbone)
indices_3d = indices_to_3d(input_indices, grid_size)

corners_3Ds = read_label(label_path)
corners_voxelaxis = corners_to_voxelaxis(corners_3Ds, coors_range_xyz, vsize_xyz)
feature_keys = get_inner_voxel(corners_voxelaxis, checker)
features = get_inner_voxel_feature(input_feature, indices_3d, feature_keys, grid_size)

In [17]:
feature_array = np.array(features)
feature_array.shape

(428, 16)

# MRMR

In [3]:
import pandas as pd
from mrmr import mrmr_classif
feature_array = pd.DataFrame(feature_array)
feature_label = pd.Series(np.ones(len(feature_array)))
selected_features = mrmr_classif(X=feature_array, y=feature_label, K=6)
selected_features

# Deepcore